### medicine_project_detector

In [1]:
# 載入套件
import numpy as np
import pandas as pd
import time
import cv2
import os
from tqdm import tqdm_notebook
import tensorflow as tf
from tensorflow import keras
from keras.applications.resnet50 import preprocess_input, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from pyzbar import pyzbar
import unicodedata

%matplotlib inline

Using TensorFlow backend.


In [2]:
# model import
import copy
import warnings
warnings.filterwarnings('ignore')
import keras
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input
from keras.layers import Conv2D, Activation, GlobalAveragePooling2D
from keras.preprocessing.image import load_img, img_to_array
import seaborn as sns
import shap
import matplotlib
import matplotlib.pylab as plt
from tensorflow.keras import callbacks

In [3]:
W = 480 # The default size for ResNet is 224 but resize to .5 to save memory size
H = 480 # The default size for ResNet is 224 but resize to .5 to save memory size
label_to_class = {
    '025866':0,
    '006271':1,
    '013382':2,
    '046613':3,
    '029090':4,
    '022105':5,
    '046118':6,
    '037441':7,
    '011701':8,
    '006263':9,
    '028452':10,
    '018620':11,
    '045267':12,
    '040011':13,
    '055232':14,
    '004085':15,
    '027940':16,
    '044710':17,
    '004928':18,
    '039863':19,
    '040699':20,
    '038575':21,
    '021882':22,
    '009102':23,
    '047418':24,
    '015392':25
}

class_to_label = {v: k for k, v in label_to_class.items()}
n_classes = len(label_to_class)

def get_images(dir_name, label_to_class):
    """read images / labels from directory"""
    
    Images = []
    Classes = []
    
    for label_name in os.listdir(dir_name):
        cls = label_to_class[label_name]
        
        for img_name in os.listdir('/'.join([dir_name, label_name])):
            img = load_img('/'.join([dir_name, label_name, img_name]), target_size=(W, H))
            img = img_to_array(img)
            
            Images.append(img)
            Classes.append(cls)
            
    Images = np.array(Images, dtype=np.float32)
    Classes = np.array(Classes, dtype=np.float32)
    Images, Classes = shuffle(Images, Classes, random_state=0)
    
    return Images, Classes

In [4]:
## get images / labels

Images, Classes = get_images(dir_name='dataset/train',label_to_class=label_to_class)
Images.shape, Classes.shape

((26, 480, 480, 3), (26,))

In [5]:
## split train / test

indices_train, indices_test = train_test_split(list(range(Images.shape[0])), train_size=0.8, test_size=0.2, shuffle=False)

x_train = Images[indices_train]
y_train = Classes[indices_train]
x_test = Images[indices_test]
y_test = Classes[indices_test]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((20, 480, 480, 3), (20,), (6, 480, 480, 3), (6,))

In [6]:
## to one-hot

y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

y_train.shape, y_test.shape

((20, 26), (6, 26))

In [7]:
## to image data generator

datagen_train = ImageDataGenerator(
    preprocessing_function=preprocess_input, # image preprocessing function
    rotation_range=30,                       # randomly rotate images in the range
    zoom_range=0.1,                          # Randomly zoom image
    width_shift_range=0.1,                   # randomly shift images horizontally
    height_shift_range=0.1,                  # randomly shift images vertically
    horizontal_flip=True,                    # randomly flip images horizontally
    vertical_flip=False,                     # randomly flip images vertically
)
datagen_test = ImageDataGenerator(
    preprocessing_function=preprocess_input, # image preprocessing function
)

In [8]:
# Resnet
input_tensor = Input(shape=(W, H, 3)) # To change input shape
resnet50 = ResNet50(
    include_top=False,                # To change output shape
    weights='imagenet',               # Use pre-trained model
    input_tensor=input_tensor,        # Change input shape for this task
)

# fc layer
top_model = Sequential()
top_model.add(GlobalAveragePooling2D())               # Add GAP for cam
top_model.add(Dense(n_classes, activation='softmax')) # Change output shape for this task

# model
model = Model(input=resnet50.input, output=top_model(resnet50.output))

# frozen weights
for layer in model.layers[:-10]:
    layer.trainable = False or isinstance(layer, BatchNormalization) # If Batch Normalization layer, it should be trainable

# compile
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
## Load訓練好的模型
model = load_model("test__resnet50.h5")

In [10]:
## barcode 檢測
import requests
from bs4 import BeautifulSoup
import re
from urllib.request import urlopen #取得pdf

def crawler_barcode_result():
    address = "rtsp://10.X.X.X:X/live"
    camera = cv2.VideoCapture(address) #選這項就是用網路相機
    #camera = cv2.VideoCapture(0) #選這項就是使用電腦相機
    ret, frame = camera.read()
    open('drugname.txt', 'w').close()
    open('drugreact.txt', 'w').close()
    open('NameCn.txt', 'w').close()
    open('drugID.txt', 'w').close()
    def read_barcodes(frame):
        barcodes = pyzbar.decode(frame)
        for barcode in barcodes:

            x, y , w, h = barcode.rect
            barcode_text = barcode.data.decode('utf-8')
            cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 255, 0), 2)

            
            #####爬蟲#####

            response = requests.get(barcode_text)
            soup = BeautifulSoup(response.text, 'lxml')
            result = soup.find('span',{'id':'lblEnName'})
            react = soup.find('span',{'id':'lblIndiCat'}).text
            NameCn = soup.find('span',{'id':'lblChName'}).text
            DrugID = soup.find('span',{'id':'lblLicName'}).text
            DrugID_num = re.findall("\d+", DrugID)[0]

#             ##這裡下面是抓取pdf檔案的部分
#             drag_main_page = urlopen(barcode_text) #获取网页
#             drag_main_page_html = BeautifulSoup(drag_main_page, 'html.parser') #解析网页
#             drag_main_page_html_str = str(drag_main_page_html)
#             drag_pageID_position = str(drag_main_page_html).find('LicId=')
#             drag_pageID = drag_main_page_html_str[drag_pageID_position+6:drag_pageID_position+14]

#             pdf_url = urlopen('https://info.fda.gov.tw/MLMS/H0001D2.aspx?LicId='+drag_pageID)
#             drag_pdf_page_html = BeautifulSoup(pdf_url, 'html.parser') #解析网页

#             pdf_page_link = drag_pdf_page_html.find_all('a')  #获取所有超链接
#             for h in pdf_page_link:
#                 pdf_hyperlink = h.get('href')

#             r = requests.get('https://info.fda.gov.tw/MLMS/'+pdf_hyperlink)
#             pdf = r.content       #響應的二進位制檔案
#             with open(drag_pageID+'.pdf','wb') as f:     #二進位制寫入
#                 f.write(pdf)


            if result==None:
                print("error")
                data=open("drugname.txt",'w+')
                print("error",file=data)
                data.close()
            else :
                result = soup.find('span',{'id':'lblEnName'}).text
                data=open("drugname.txt",'w+')
                print(result,file=data)
                data.close()
                data_rec=open("drugreact.txt",'w+')
                print(react,file=data_rec)
                data_rec.close()
                data_Cn=open("NameCn.txt",'w+')
                print(NameCn,file=data_Cn)
                data_Cn.close()
                data_ID=open("drugID.txt",'w+')
                print(DrugID_num,file=data_ID)
                data.close()

        return frame
    while ret:

        ret, frame = camera.read()
        frame = read_barcodes(frame)
        #cv2.imshow('Barcode reader', frame)


        if os.stat("drugID.txt").st_size != 0:
            s = requests.session()
            s.keep_alive = False
            break
        if cv2.waitKey(1) & 0xFF == 27:
            break

    camera.release()
    cv2.destroyAllWindows()

In [11]:
### 影像讀取
def image_result():
    cam2 = cv2.VideoCapture("rtsp://10.X.X.X:X/live")
    ticks_begin = time.time()
    while True:
        ret, img = cam2.read()
        vis = img.copy()
        medicine_img = img
        ticks_medium = time.time()
        if ticks_medium-ticks_begin>=2:
            break
        if 0xFF & cv2.waitKey(2) == 27:
            break
    #print("只給你5秒讀取:",ticks_medium-ticks_begin)
    cv2.imwrite('dataset/testset/025866/000001.jpg', img)
# cam2 顯示影像
# cv2.imshow('this what you watching!!', medicine_img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [12]:
def imageform_generate():
    ## test資料轉換為ImageDataGenerator格式, 比照train/validation data logging
    RImages, RClasses = get_images(dir_name='dataset/testset',label_to_class=label_to_class)
    RImages.shape, RClasses.shape
    
    ## real image
    x_real = RImages
    y_real = RClasses
    
    ## to one-hot
    y_real = keras.utils.to_categorical(y_real, n_classes)
    
    ## to image data generator
    datagen_real = ImageDataGenerator(
    preprocessing_function=preprocess_input,) # image preprocessing function
    
    ## predict image result
    x = preprocess_input(copy.deepcopy(x_real))
    y_preds = model.predict(x).tolist()[0]
    #print('y_preds:',y_preds)

    ## transfer to the medicine name
    class_number = y_preds.index(max(y_preds))
    #print('class_number:',class_number)
    labels = dict((k,v) for k,v in class_to_label.items())
    #print('labels:',labels)
    
    predictions = class_to_label[int(class_number)]
    #print('predictions:',predictions)
    return predictions

In [13]:
# 轉成ascii碼型式
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)\
        if unicodedata.category(c) != 'Mn')

In [14]:
def detector():
    crawler_barcode_result()
    image_result()
    image_class = imageform_generate()
    detector_classname = drugname_to_class.get(image_class)  #偵測到的藥品名稱
    print('辨便Bang影片判斷結果為:',image_class, detector_classname)
    
    with open("drugID.txt", "r") as f:    #開啟檔案
        result = f.read()   #讀取檔案
        
    print('食藥署QRCODE爬蟲結果為:',result)
    result=result[0:6]
    if result==image_class:
        ans=1
        #print("Medicine class is true!!")
    elif result!=image_class:
        ans=0
        #print("Medicine class is false!!")
    elif result==None:
        ans=9
    return ans, result, image_class

In [ ]:
import tkinter as tk
from PIL import Image, ImageTk

def close_window(): #######################
    window.destroy()  #######################

drugname_to_class = {
'025866': 'Amoxicillin',
'006271': 'Fucole PARAN',
'013382': 'Magnesium Oxide',
'046613': 'U-Chu Tonec',
'029090': 'TonFul',
'022105': 'Ulexin',
'046118': 'B.H.L',
'037441': 'Biperin',
'011701': 'Peiwetsu',
'006263': 'Bismuth',
'028452': 'Buscopan',
'018620': 'Undiarrhea',
'045267': 'U-Chuaceo',
'040011': 'Acemet',
'055232': 'KetenE.M.C.',
'004085': 'Alinamin-F50',
'027940': 'Beesix',
'044710': 'CetyFilm',
'004928': 'DailyCare',
'039863': 'Gincare',
'040699': 'Ginkgocentrate',
'038575': 'MabalCapsules',
'021882': 'Ningilon',
'009102': 'Propranolol',
'047418': 'Suride',
'015392': 'Transamin'

}

try_max_length=100000 #100000次
i=0
while i < try_max_length+1:
    print('[[機器判斷中...請耐心等待]]'+'\n')
    ans, drugname, image_class = detector() # 正確/錯誤, 食藥署number, 影像辨識結果number
    
    with open("drugreact.txt", "r") as f:    #開啟檔案
        effectiveness_new = f.read()   #讀取檔案
    classname = drugname_to_class.get(drugname) #以食藥署number反查藥品名稱, 有\n要小心, 只保留前6碼
    
    max_txt_len = 15
    word_number = 1+len(effectiveness_new)//15
    if word_number == 1:
        contain = classname +"\n用途 : "+effectiveness_new[:]
    elif word_number == 2:
        contain = classname +"\n用途 : "+effectiveness_new[0:max_txt_len]+'\n'+effectiveness_new[max_txt_len:] 
    elif word_number == 3:
        contain = classname +"\n用途 : "+effectiveness_new[0:max_txt_len]+'\n'+effectiveness_new[max_txt_len:2*max_txt_len]+'\n'+effectiveness_new[2*max_txt_len:] 
    elif word_number == 4:
        contain = classname +"\n用途 : "+effectiveness_new[0:max_txt_len]+'\n'+effectiveness_new[max_txt_len:2*max_txt_len]+'\n'+effectiveness_new[2*max_txt_len:3*max_txt_len] 
    
    if ans==0:
        window = tk.Tk()
        window.title('藥物辨識')
        window.wm_attributes('-topmost',1) #視窗出現在最上層
        window.geometry("620x500+350+120") #視窗出現位置跟大小
        label = tk.Button(window,                 # 文字標示所在視窗
                        text = '\n(X) 藥 品 錯 誤!\n正確藥品應為 : '+ contain,  # 顯示文字
                         bg = 'white',         #  背景顏色
                         fg = 'red', 
                         font = ('arial', 20),   # 字型與大小
    #                      width = 80, height = 3) # 文字標示尺寸   
                         width = 40, height = 5,command=close_window) ####################
        label.pack()

        img = ImageTk.PhotoImage(Image.open("./ng.jpg").resize((310, 310), Image.ANTIALIAS))  # NG圖
        canvas = tk.Canvas(window, height=620, width=620)
        image = canvas.create_image(155, 0, anchor='n',image=img) 
        canvas.pack()

        im_Med = Image.open("./" + classname + ".png") #藥物圖
        im_Med = im_Med.resize((310, 310), Image.ANTIALIAS)
        img_Med_re = ImageTk.PhotoImage(im_Med)
        image_ = canvas.create_image(310+155, 0, anchor='n',image=img_Med_re)

        window.mainloop()


    elif ans==1:
        window = tk.Tk()
        window.title('藥物辨識')
        window.wm_attributes('-topmost',1)
        window.geometry("620x500+350+120") #視窗出現位置跟大小
        label = tk.Button(window,                 # 文字標示所在視窗
                         text = '\n(O) 藥 品 符 合 !\n'+ contain,  # 顯示文字
                         bg = 'white',         #  背景顏色
                         fg = '#228B22', 
                         font = ('arial', 20),   # 字型與大小
    #                      width = 80, height = 3,) # 文字標示尺寸   
                         width = 40, height = 5,command=close_window) ####################
        label.pack()

        img = ImageTk.PhotoImage(Image.open("./ok.jpeg").resize((310, 310), Image.ANTIALIAS))  # NG圖
        canvas = tk.Canvas(window, height=620, width=620)
        image = canvas.create_image(155, 0, anchor='n',image=img) 

        im_Med = Image.open("./" + classname + ".png") #藥物圖
        im_Med = im_Med.resize((310, 310), Image.ANTIALIAS)
        img_Med_re = ImageTk.PhotoImage(im_Med)
        image_ = canvas.create_image(310+165, 0, anchor='n',image=img_Med_re)

        canvas.pack()

        window.mainloop()


    else:

        window = tk.Tk()
        window.title('藥物辨識')
        window.wm_attributes('-topmost',1)
        window.geometry("256x40+350+250") #視窗出現位置跟大小
        label = tk.Label(window,                 # 文字標示所在視窗
                         text = '\n非 預 期 錯 誤 !',  # 顯示文字
                         bg = '#F0E68C',         #  背景顏色
                         fg = '#C71585', 
                         font = ('arial', 20),   # 字型與大小
                         width = 15, height = 2) # 文字標示尺寸   
    #                      width = 15, height = 2,command=close_window) ####################
        label.pack()

        window.mainloop()

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 046118 B.H.L
食藥署QRCODE爬蟲結果為: 046118

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 022105 Ulexin
食藥署QRCODE爬蟲結果為: 022105

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 018620 Undiarrhea
食藥署QRCODE爬蟲結果為: 055232

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 018620 Undiarrhea
食藥署QRCODE爬蟲結果為: 018620

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 025866 Amoxicillin
食藥署QRCODE爬蟲結果為: 025866

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 027940 Beesix
食藥署QRCODE爬蟲結果為: 047418

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 046118 B.H.L
食藥署QRCODE爬蟲結果為: 037441

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 028452 Buscopan
食藥署QRCODE爬蟲結果為: 028452

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 006271 Fucole PARAN
食藥署QRCODE爬蟲結果為: 006271

[[機器判斷中...請耐心等待]]

辨便Bang影片判斷結果為: 006263 Bismuth
食藥署QRCODE爬蟲結果為: 006263

